In [4]:
import xarray as xr
import numpy as np
import pandas as pd

ds = xr.open_dataset("../ssp126-outputs.nc")

tas = ds["tas"]  # [lon,lat,time,member]
lat = ds["lat"]
lon = ds["lon"]

# area weights
weights = np.cos(np.deg2rad(lat))
weights = weights / weights.sum()

# global mean time series (ensemble member 0)
tas_global = (tas.isel(member=0)
                .weighted(weights)
                .mean(dim=["lon","lat"]))

# convert to pandas
df = tas_global.to_dataframe().reset_index()

# compute anomaly relative to first 30 years
baseline = df["tas"].iloc[:30].mean()
df["tas_anom"] = df["tas"] - baseline

df.head()


,time,quantile,member,tas,tas_anom
0,2015,0.9,1,0.821611,-0.253328
1,2016,0.9,1,0.817080,-0.257859
2,2017,0.9,1,0.928603,-0.146335
3,2018,0.9,1,0.980651,-0.094287
4,2019,0.9,1,0.921126,-0.153813


In [6]:
import subprocess

subprocess.run([
    "/usr/local/bin/Rscript",
    "../BRICK/run_brick_custom.R",
    "projections/ssp126_tas_projections.csv"
])


Error in library(brick) : there is no package called ‘brick’
Execution halted


CompletedProcess(args=['/usr/local/bin/Rscript', '../BRICK/run_brick_custom.R', 'projections/ssp126_tas_projections.csv'], returncode=1)